In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
STREET = 'washington'

df = pd.read_csv(f'data/{STREET}_merged.csv')

df['time'] = pd.to_datetime(df['time'])

print("Number of rows: ", len(df))
print("Columns: ", df.columns)

print("Start date: ", df['time'].min())
print("End date: ", df['time'].max())

In [ ]:
df.head(10)

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
fig = px.line(df, x='time', y='throughput', title='Throughput Over Time')
fig.show()

In [ ]:
fig = px.line(df, x='time', y='occupancy', title='Occupancy Over Time')
fig.show()

In [ ]:
# Example mappings (update based on actual definitions)
closed_state_mapping = {0: 'Unknown', 1: 'Open', 2: 'Closed', 3: 'Invalid'}
traffic_state_mapping = {0: 'Unknown', 1: 'Fluid', 2: 'Pre-saturated', 3: 'Saturated', 4: 'Blocked'}

tmp_df = df.copy()
tmp_df['closed_state_desc'] = tmp_df['closed_state'].map(closed_state_mapping)
tmp_df['traffic_state_desc'] = tmp_df['traffic_state'].map(traffic_state_mapping)

fig = px.scatter(tmp_df, x='time', y='throughput', color='traffic_state_desc', title='Throughput Over Time by Traffic State')
fig.show()

fig = px.scatter(tmp_df, x='time', y='occupancy', color='traffic_state_desc', title='Occupancy Over Time by Traffic State')
fig.show()

In [ ]:
fig = px.scatter(tmp_df, x='time', y='throughput', color='closed_state_desc', title='Throughput Over Time by Closing State')
fig.show()

fig = px.scatter(tmp_df, x='time', y='occupancy', color='closed_state_desc', title='Occupancy Over Time by Closing State')
fig.show()